In [48]:
import pandas as pd
import numpy as np
import re
import spacy

In [49]:
from spacy.matcher import PhraseMatcher

In [50]:
# import spacy.cli
# spacy.cli.download("en_core_web_lg")
## ref: https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon

In [51]:
# This only happens to Ying that she couldn't import 'en_core_web_sm' in Jupyter Notebook, so she imports it with the full path
nlp = spacy.load("/usr/local/Cellar/jupyterlab/3.2.9/libexec/lib/python3.9/site-packages/en_core_web_lg/en_core_web_lg-3.2.0/")

In [52]:
# terms would be each list in the dict

In [53]:

# import the dataset 

file1 = '2020_FB_Pres_Ads_500_Topic_Gold_v1_test.xlsx'
file2 = '2020_FB_Pres_Posts_500_Topic_GOLD_v1_test.xlsx'
file3 = '2020_TW_Pres_Posts_500_Topic_Gold_v1_test.xlsx'
dirs = 'C:\\Users\\wangtao\\Documents\\Illumination\\data_set(2020july)\\'

# df1 = pd.read_excel(dirs+file3)

df1 = pd.read_csv('FBtest_1500.csv') # here we import combined the test dataset
df1.head(20)

,text,Gold Label
0,When President Obama needed a champion for wor...,no topic
1,When President Obama needed a champion for wor...,no topic
2,President Obama called Elizabeth “one of the c...,economic
3,Elizabeth isn’t afraid to stand up to billiona...,no topic
4,The Nevada caucuses are almost here! \n\nThere...,no topic
5,The Nevada caucuses are almost here! There’s a...,no topic
6,I’m not afraid to stand up to billionaires and...,no topic
7,President Obama called Elizabeth “one of the c...,economic
8,I’m so grateful for the support of this grassr...,no topic
9,"Last quarter, Bernie Sanders, Pete Buttigieg a...",no topic


In [54]:
import json 
with open ('2020_july_lexiconV4_7.json') as f1: # import the latest lexicon
    dic1 = json.load(f1)
# check the keys
dic1.keys()

dict_keys(['covid', 'economic', 'education', 'environment', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social and cultural', 'social programs'])

In [55]:
# The column that contains the text should be uniformly renamed as "text" 
df1.rename(columns={'ad_creative_body':'text'}, inplace=True)
df1

,text,Gold Label
0,When President Obama needed a champion for wor...,no topic
1,When President Obama needed a champion for wor...,no topic
2,President Obama called Elizabeth “one of the c...,economic
3,Elizabeth isn’t afraid to stand up to billiona...,no topic
4,The Nevada caucuses are almost here! \n\nThere...,no topic
...,...,...
1495,Yo invertiría en infraestructura para que pers...,no topic
1496,"“During his Wisconsin out-reach, it became cle...",no topic
1497,Another endorsement coming out of Iowa today! ...,no topic
1498,Wow! Thanks everyone for supporting Help Meado...,no topic


In [56]:
#df1[df1['text'].isnull()]

In [57]:
# check the dataset 
if df1['text'].isnull().sum():
    print (df1['text'].isnull().sum())
    df1 = df1[df1['text'].notnull()]
    df1.reset_index(drop=True,inplace=True)

1


In [58]:
df1 # check the dataset

,text,Gold Label
0,When President Obama needed a champion for wor...,no topic
1,When President Obama needed a champion for wor...,no topic
2,President Obama called Elizabeth “one of the c...,economic
3,Elizabeth isn’t afraid to stand up to billiona...,no topic
4,The Nevada caucuses are almost here! \n\nThere...,no topic
...,...,...
1494,Yo invertiría en infraestructura para que pers...,no topic
1495,"“During his Wisconsin out-reach, it became cle...",no topic
1496,Another endorsement coming out of Iowa today! ...,no topic
1497,Wow! Thanks everyone for supporting Help Meado...,no topic


In [59]:
# function that filter the urls and symbols in the text 
def filter_text(x):
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s]",' ',x) # filter symbols
    x = re.sub("\s+",' ',x)
    
    ls=[w.lower() for w in x.split()] 
    
    return ' '.join(ls)

In [60]:
df1['text'] = df1['text'].astype(str).apply(lambda x: filter_text(x))
df1

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_468/2351442672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['text'] = df1['text'].astype(str).apply(lambda x: filter_text(x))


,text,Gold Label
0,when president obama needed a champion for wor...,no topic
1,when president obama needed a champion for wor...,no topic
2,president obama called elizabeth one of the co...,economic
3,elizabeth isn t afraid to stand up to billiona...,no topic
4,the nevada caucuses are almost here there s a ...,no topic
...,...,...
1494,yo invertiría en infraestructura para que pers...,no topic
1495,during his wisconsin out reach it became clear...,no topic
1496,another endorsement coming out of iowa today l...,no topic
1497,wow thanks everyone for supporting help meadow...,no topic


In [61]:
# import matcher
from spacy.matcher import Matcher 

In [62]:
# # Ying checks if nlp works
# words = df1['text'].astype(str).tolist()
# doc = nlp(words[1])
# doc

In [63]:
# #Ying tries to understand how token works
# print([token.text for token in doc])

In [64]:
# topics = dic1.keys()
# for t in topics:
#     print(dic1[t])

In [65]:
# topics = dic1.keys()
# for t in topics:
#     matcher = Matcher(nlp.vocab)
#     terms = dic1[t]
#     for text in terms:
#         patterns = nlp.make_doc(text)
#         print(patterns)

In [66]:
# topics = dic1.keys()
# for t in topics:
#     matcher = Matcher(nlp.vocab)
#     terms = dic1[t]
#     patterns = [nlp.make_doc(text) for text in terms]
#     matcher.add('TerminologyList',[patterns])
#     print(pattern)

In [67]:
# the function that find the lexicon words in the text
def find_words(x,lexicon):
    topics= lexicon.keys()  
    doc = nlp(x) # nlp() is spaCy 2.2 English language model 
    words= []
    for t in topics:
        matcher = Matcher(nlp.vocab)
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        print(patterns)
        matcher.add("TerminologyList", [patterns]) # spaCy2.2 phrase matcher
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    if words:
        words = list(set(words))
        return ','.join(words)
    else:
        return('no words')
        

In [68]:
# tagging the topic in each message
def find_topic(x,lexicon):
    topics= lexicon.keys()    
    if x=='no words':
        return ''    
    if x != 'no words': 
        words = x.split(',')
        labels = []        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                labels.append(t)                
                #l = sorted(labels)        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [69]:
df1['words'] = df1['text'].astype(str).apply(lambda x: find_words(x,dic1))

[caronavirus, bed capacity, see collapse economy, panicbuying, evacuee, social distancing, panicshop, ncov, protect working people, world health organization, ppe, mers, sflockdown, national testing strategy, china virus, distancing, ventilators, 14dayquarantine, quarantinelife, chloroquine, chinesevirus, masks, quarantine, ebola, protective equipment, panic shopping, duringmy14dayquarantine, canceleverything, flatten the curve, stay at home, pandemic, wet markets, covidiot, panic shop, shelteringinplace, inmyquarantinesurvivalkit, kungflu, koronavirus, chinavirus, liberate, isolating, ppeshortage, stay home, going see collapse, coronavirus, corona virus, virus, remdesivir, panic buy, corona, sheltering in place, disinfect, flu, n95, quarentinelife, trumppandemic, covid19, stayhomechallenge, screening, nonessential, disinfectants, infuenza, coronavirus pandemic, outbreak, trump pandemic, stayhome, infected patients, restrict travel, vaccine, sinophobia, pandemie, wuhancoronavirus, getm

MatchPatternError: Invalid token patterns for matcher rule 'TerminologyList'

Pattern 0:
- [pattern -> 0] value is not a valid dict
- [pattern -> 1] value is not a valid dict
- [pattern -> 2] value is not a valid dict
- [pattern -> 3] value is not a valid dict
- [pattern -> 4] value is not a valid dict
- [pattern -> 5] value is not a valid dict
- [pattern -> 6] value is not a valid dict
- [pattern -> 7] value is not a valid dict
- [pattern -> 8] value is not a valid dict
- [pattern -> 9] value is not a valid dict
- [pattern -> 10] value is not a valid dict
- [pattern -> 11] value is not a valid dict
- [pattern -> 12] value is not a valid dict
- [pattern -> 13] value is not a valid dict
- [pattern -> 14] value is not a valid dict
- [pattern -> 15] value is not a valid dict
- [pattern -> 16] value is not a valid dict
- [pattern -> 17] value is not a valid dict
- [pattern -> 18] value is not a valid dict
- [pattern -> 19] value is not a valid dict
- [pattern -> 20] value is not a valid di

In [70]:
df1['m_label'] = df1['words'].apply(lambda x: find_topic(x,dic))

KeyError: 'words'

In [71]:
df1['m_label'] = df1['m_label'].apply(lambda x: 'no topic' if x=='' else x)

KeyError: 'm_label'